In [118]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


In [119]:
df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\processed\modelo_dataset.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28894 entries, 0 to 28893
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  28894 non-null  int64  
 1   original_language   28894 non-null  object 
 2   overview            28894 non-null  object 
 3   popularity          28894 non-null  float64
 4   revenue             28894 non-null  float64
 5   runtime             28894 non-null  object 
 6   title               28894 non-null  object 
 7   vote_average        28894 non-null  float64
 8   vote_count          28894 non-null  int64  
 9   release_year        28894 non-null  int64  
 10  btc_name            3620 non-null   object 
 11  combined_companies  28894 non-null  object 
 12  predictor           28894 non-null  object 
 13  generos             28894 non-null  object 
 14  actores             28295 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 3.3+

In [141]:
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(min_df=3, max_df=0.85, max_features=40000, dtype=np.float32) 

# Transformar la columna 'overview' en una matriz TF-IDF
matriz = vectorizer.fit_transform(df['predictor'])

# Reductir la dimensionalidad con SVD.
svd = TruncatedSVD(n_components=700, random_state=42)

matriz_reducida = svd.fit_transform(matriz)
num_features = len(vectorizer.vocabulary_)

print(f"El vectorizador está utilizando {num_features} características.")
print(f"Dimensiones reducidas a {matriz_reducida.shape[1]} componentes.")

El vectorizador está utilizando 40000 características.
Dimensiones reducidas a 700 componentes.


In [142]:
# Calcular la matriz de similitud del coseno
cosine_sim = cosine_similarity(matriz_reducida, matriz_reducida)

In [145]:
def recommendacion(title, cosine_sim=cosine_sim):
    # Encuentra el índice de la película
    idx = df.index[df['title'] == title].tolist()
    if not idx:
        return "Película no encontrada"
    idx = idx[0]
    
    # Obtén los puntajes de similitud para la película seleccionada
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordena las películas basadas en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtén los índices de las películas más similares
    movie_indices = [i[0] for i in sim_scores[1:6]]  # 10 películas más similares
    
    return df['title'].iloc[movie_indices]

# Prueba la función de recomendación
print(recommendacion('despicable me 3'))

21760              the secret life of pets
16244                      despicable me 2
4113                        monsters, inc.
12663                        despicable me
11767    cloudy with a chance of meatballs
Name: title, dtype: object


In [123]:
# Despicable me 3
# 
